In [11]:
import simpy
import random
import numpy as np

# Define constants based on the provided parameters
NUM_CLIENTS = random.randint(100, 200)
NUM_DATACENTERS = 1
NUM_HOSTS = 2

# Cloud node parameters
CLOUD_NODES = 10
CLOUD_CPU_POWER = (3000, 5000)  # MIPS
CLOUD_RAM = 8  # GB
CLOUD_STORAGE = 1  # TB
CLOUD_BANDWIDTH = 0.5  # Gb/s

# Fog node parameters
FOG_NODES = 10
FOG_CPU_POWER = (1000, 2800)  # MIPS
FOG_RAM = 2  # GB
FOG_STORAGE = 0.5  # TB
FOG_BANDWIDTH = 1  # Gb/s

# Task parameters
TASK_LENGTH = (2000, 56000)  # MI
DATA_SIZE = (400, 600)  # MB
TASK_SETS = [300, 600, 900, 1200, 1500]  # Different sets of task numbers

class Node:
    def __init__(self, env, cpu_power, ram, storage, bandwidth):
        self.env = env
        self.cpu_power = cpu_power
        self.ram = ram
        self.storage = storage
        self.bandwidth = bandwidth
        self.queue = simpy.Resource(env, capacity=1)
        self.total_execution_time = 0

    def process_task(self, task_length, data_size):
        # Processing time is based on task length and CPU power
        processing_time = task_length / self.cpu_power
        self.total_execution_time += processing_time
        yield self.env.timeout(processing_time)

    def calculate_energy_consumption(self, makespan):
        beta_j = 10**-8 * self.cpu_power**2
        alpha_j = 0.6 * beta_j
        active_energy = self.total_execution_time * beta_j
        idle_energy = (makespan - self.total_execution_time) * alpha_j
        total_energy = active_energy + idle_energy
        return total_energy

def generate_tasks(env, nodes, task_length_range, data_size_range, num_tasks, task_allocation):
    task_start_times = []
    task_end_times = []
    
    for task_id in range(num_tasks):
        task_length = random.randint(*task_length_range)
        data_size = random.randint(*data_size_range)
        node = nodes[task_allocation[task_id]]
        task_start_times.append(env.now)
        env.process(handle_task(env, node, task_length, data_size, task_end_times))
        yield env.timeout(1)  # Small delay to simulate task arrival interval
    
    yield env.timeout(0)  # Yield control to allow all tasks to be processed
    yield env.all_of([env.timeout(1) for _ in range(num_tasks)])  # Wait for all tasks to be completed
    makespan = max(task_end_times) - min(task_start_times)
    return makespan

def handle_task(env, node, task_length, data_size, task_end_times):
    with node.queue.request() as request:
        yield request
        #print(f"Task assigned to node with CPU power {node.cpu_power} at time {env.now}")
        yield env.process(node.process_task(task_length, data_size))
        task_end_times.append(env.now)

def fitness_function(env, all_nodes, num_tasks, task_allocation):
    makespan = env.process(generate_tasks(env, all_nodes, TASK_LENGTH, DATA_SIZE, num_tasks, task_allocation))
    env.run()
    makespan = makespan.value
    total_energy_consumption = sum(node.calculate_energy_consumption(makespan) for node in all_nodes)
    return makespan, total_energy_consumption

def qpso_optimize(num_particles, num_iterations, num_tasks, num_nodes):
    # Initialize particles
    particles = np.random.randint(0, num_nodes, (num_particles, num_tasks))
    pbest_positions = particles.copy()
    pbest_scores = np.full(num_particles, np.inf)
    gbest_position = None
    gbest_score = np.inf
    
    for _ in range(num_iterations):
        for i in range(num_particles):
            env = simpy.Environment()
            cloud_nodes = [
                Node(env, random.randint(*CLOUD_CPU_POWER), CLOUD_RAM, CLOUD_STORAGE, CLOUD_BANDWIDTH)
                for _ in range(CLOUD_NODES)
            ]
            fog_nodes = [
                Node(env, random.randint(*FOG_CPU_POWER), FOG_RAM, FOG_STORAGE, FOG_BANDWIDTH)
                for _ in range(FOG_NODES)
            ]
            all_nodes = cloud_nodes + fog_nodes
            
            makespan, energy_consumption = fitness_function(env, all_nodes, num_tasks, particles[i])
            score = makespan + energy_consumption
            
            if score < pbest_scores[i]:
                pbest_scores[i] = score
                pbest_positions[i] = particles[i].copy()
                
            if score < gbest_score:
                gbest_score = score
                gbest_position = particles[i].copy()
        
        mb = np.mean(pbest_positions, axis=0)
        beta = 1.0  # You can adjust this parameter based on your problem
        u = np.random.rand(num_particles, num_tasks)
        particles = mb + beta * np.abs(pbest_positions - particles) * np.log(1.0 / u)
        
    return gbest_position, gbest_score

def main():
    results = {}
    num_particles = 100
    num_iterations = 500

    for num_tasks in TASK_SETS:
        gbest_position, gbest_score = qpso_optimize(num_particles, num_iterations, num_tasks, CLOUD_NODES + FOG_NODES)
        
        env = simpy.Environment()
        cloud_nodes = [
            Node(env, random.randint(*CLOUD_CPU_POWER), CLOUD_RAM, CLOUD_STORAGE, CLOUD_BANDWIDTH)
            for _ in range(CLOUD_NODES)
        ]
        fog_nodes = [
            Node(env, random.randint(*FOG_CPU_POWER), FOG_RAM, FOG_STORAGE, FOG_BANDWIDTH)
            for _ in range(FOG_NODES)
        ]
        all_nodes = cloud_nodes + fog_nodes
        
        makespan, energy_consumption = fitness_function(env, all_nodes, num_tasks, gbest_position)
        
        results[num_tasks] = {
            "makespan": makespan,
            "energy_consumption": energy_consumption
        }

    # Output the makespan and energy consumption for each set of tasks
    for num_tasks, result in results.items():
        print(f"Makespan for {num_tasks} tasks: {result['makespan']}")
        print(f"Total energy consumption for {num_tasks} tasks: {result['energy_consumption']}")

if __name__ == "__main__":
    main()


Task assigned to node with CPU power 4503 at time 0
Task assigned to node with CPU power 3562 at time 1
Task assigned to node with CPU power 4946 at time 2
Task assigned to node with CPU power 1467 at time 4
Task assigned to node with CPU power 4503 at time 5
Task assigned to node with CPU power 4503 at time 6.3704197201865425
Task assigned to node with CPU power 3780 at time 7
Task assigned to node with CPU power 3562 at time 8
Task assigned to node with CPU power 1908 at time 10
Task assigned to node with CPU power 2333 at time 11
Task assigned to node with CPU power 1430 at time 12
Task assigned to node with CPU power 4946 at time 12.059239789729075
Task assigned to node with CPU power 1908 at time 13
Task assigned to node with CPU power 2133 at time 14
Task assigned to node with CPU power 3466 at time 15
Task assigned to node with CPU power 3287 at time 16
Task assigned to node with CPU power 4503 at time 17
Task assigned to node with CPU power 4241 at time 18
Task assigned to node

TypeError: list indices must be integers or slices, not numpy.float64